In [ ]:
# @title 🎬 ГОДЖО 30 СЕК — ПРОСТАЯ ВЕРСИЯ БЕЗ COMFYUI
import os
import subprocess
import time
from IPython.display import FileLink, display

# ============================================
# 📝 НАСТРОЙКИ - МЕНЯЙ ТОЛЬКО ЭТО!
# ============================================
PROMPT = "cinematic portrait of gojo satoru, white spiky hair, black blindfold, confident expression, anime style, highly detailed, 8k, professional lighting"
NEGATIVE_PROMPT = "blurry, deformed, low quality, watermark, text, bad anatomy, multiple heads, duplicate"

# Режим работы
USE_ANIMATEDIFF = True  # True = настоящая анимация, False = статичные кадры + RIFE

# Параметры генерации
WIDTH = 512
HEIGHT = 768
NUM_FRAMES = 16 if USE_ANIMATEDIFF else 8  # AnimateDiff: 16-24, обычный: 8-12
STEPS = 25 if USE_ANIMATEDIFF else 20  # Для анимации нужно больше steps
CFG_SCALE = 7.5 if USE_ANIMATEDIFF else 7
FPS = 8  # FPS для финального видео

# Интерполяция RIFE (опционально)
USE_RIFE = True  # Применить RIFE для ещё более плавной анимации
RIFE_EXP = 4 if USE_ANIMATEDIFF else 5  # AnimateDiff: 4 (16→256), обычный: 5 (8→256)
# ============================================

# === ПАПКИ ===
WORKSPACE = "/kaggle/working"
FRAMES_DIR = f"{WORKSPACE}/frames"
DATASET_DIR = "/kaggle/input/comfyui-models-gojo"

os.makedirs(FRAMES_DIR, exist_ok=True)

# === УСТАНОВКА ЗАВИСИМОСТЕЙ (ОДИН РАЗ) ===
print("🔧 Установка зависимостей...")

# Устанавливаем все сразу, чтобы избежать проблем с импортом
import sys
import importlib.util

def check_package(package_name):
    """Быстрая проверка наличия пакета без полного импорта"""
    return importlib.util.find_spec(package_name) is not None

# Список необходимых пакетов
packages_to_install = []

if not check_package("diffusers"):
    packages_to_install.append("diffusers[torch]")
    packages_to_install.append("transformers")
    packages_to_install.append("accelerate")
else:
    print("✓ diffusers уже установлен")

if not check_package("cv2"):
    packages_to_install.append("opencv-python")
else:
    print("✓ opencv-python уже установлен")

if USE_ANIMATEDIFF and not check_package("imageio"):
    packages_to_install.append("imageio")
    packages_to_install.append("imageio-ffmpeg")
elif USE_ANIMATEDIFF:
    print("✓ imageio уже установлен")

# Устанавливаем все пакеты одной командой
if packages_to_install:
    print(f"Установка: {', '.join(packages_to_install)}...")
    !pip install -q {' '.join(packages_to_install)}
    print("✓ Установка завершена!")

print("✓ Зависимости готовы!\n")

# === ГЕНЕРАЦИЯ КАДРОВ / АНИМАЦИИ ===
print(f"🎨 {'Генерация анимации' if USE_ANIMATEDIFF else 'Генерация кадров'} ({NUM_FRAMES} кадров)...")
print(f"   Промпт: {PROMPT[:80]}...")
print(f"   Размер: {WIDTH}x{HEIGHT}, Steps: {STEPS}, CFG: {CFG_SCALE}\n")

# Импортируем библиотеки (это может занять ~30 сек при первом запуске)
print("Загрузка PyTorch и PIL...")
import torch
from PIL import Image
print("✓ Библиотеки загружены")

if USE_ANIMATEDIFF:
    # === РЕЖИМ ANIMATEDIFF - НАСТОЯЩАЯ АНИМАЦИЯ ===
    from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
    from diffusers.utils import export_to_video

    print("Загрузка AnimateDiff pipeline...")

    # Загружаем motion adapter
    adapter = MotionAdapter.from_pretrained(
        "guoyww/animatediff-motion-adapter-v1-5-2",
        torch_dtype=torch.float16
    )

    # AnimateDiff работает с SD 1.5
    pipe = AnimateDiffPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        motion_adapter=adapter,
        torch_dtype=torch.float16
    ).to("cuda")

    pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
    pipe.enable_vae_slicing()
    pipe.enable_model_cpu_offload()

    print("✓ AnimateDiff готов!\n")
    print("Генерация анимации (это займет ~3-7 минут)...")

    start_time = time.time()

    # Генерируем анимацию
    output = pipe(
        prompt=PROMPT,
        negative_prompt=NEGATIVE_PROMPT,
        num_frames=NUM_FRAMES,
        width=WIDTH,
        height=HEIGHT,
        num_inference_steps=STEPS,
        guidance_scale=CFG_SCALE,
        generator=torch.Generator("cuda").manual_seed(42)
    )

    frames = output.frames[0]
    total_gen_time = time.time() - start_time

    print(f"✅ Анимация готова за {total_gen_time:.1f}s!\n")

    # Сохраняем кадры
    for i, frame in enumerate(frames):
        frame.save(f"{FRAMES_DIR}/{i}.png")

    # Создаем базовое видео
    base_video = f"{WORKSPACE}/ANIMATED_BASE.mp4"
    export_to_video(frames, base_video, fps=FPS)
    print(f"✓ Базовое видео сохранено ({len(frames)} кадров, {FPS} fps)")

    del pipe, adapter
    torch.cuda.empty_cache()

else:
    # === РЕЖИМ СТАТИЧНЫХ КАДРОВ ===
    from diffusers import StableDiffusionXLPipeline

    # Загружаем модель из датасета или HuggingFace
    model_path = f"{DATASET_DIR}/sd_xl_base_1.0.safetensors"

    if not os.path.exists(model_path):
        print(f"⚠️ Модель не найдена в датасете, используем HuggingFace...")
        model_path = "stabilityai/stable-diffusion-xl-base-1.0"

    print("Загрузка модели SDXL...")
    pipe = StableDiffusionXLPipeline.from_single_file(
        model_path,
        torch_dtype=torch.float16,
        use_safetensors=True
    ).to("cuda")

    pipe.enable_attention_slicing()
    pipe.enable_vae_slicing()

    print("✓ Модель загружена!\n")

    # Генерируем кадры
    start_time = time.time()
    for i in range(NUM_FRAMES):
        print(f"Кадр {i+1}/{NUM_FRAMES}...", end=" ")

        generator = torch.Generator(device="cuda").manual_seed(42 + i)

        image = pipe(
            prompt=PROMPT,
            negative_prompt=NEGATIVE_PROMPT,
            width=WIDTH,
            height=HEIGHT,
            num_inference_steps=STEPS,
            guidance_scale=CFG_SCALE,
            generator=generator
        ).images[0]

        image.save(f"{FRAMES_DIR}/{i}.png")
        print(f"✓ ({time.time() - start_time:.1f}s)")

    total_gen_time = time.time() - start_time
    print(f"\n✅ {NUM_FRAMES} кадров за {total_gen_time:.1f}s!\n")

    del pipe
    torch.cuda.empty_cache()

# === UPSCALE С REAL-ESRGAN (ОПЦИОНАЛЬНО) ===
print("\n📈 Апскейл кадров с Real-ESRGAN...")

# Проверяем есть ли upscale модель
upscale_model = f"{DATASET_DIR}/4x-UltraSharp.pth"

if os.path.exists(upscale_model):
    # Клонируем Real-ESRGAN если еще не установлен
    if not os.path.exists(f"{WORKSPACE}/Real-ESRGAN"):
        !git clone https://github.com/xinntao/Real-ESRGAN {WORKSPACE}/Real-ESRGAN
        %cd {WORKSPACE}/Real-ESRGAN
        !pip install -q basicsr facexlib gfpgan
        !pip install -q realesrgan

    %cd {WORKSPACE}/Real-ESRGAN

    # Копируем модель
    !mkdir -p weights
    !cp {upscale_model} weights/

    # Апскейл каждого кадра
    print("Запуск апскейла...")
    !python inference_realesrgan.py \
        -n 4x-UltraSharp \
        -i {FRAMES_DIR} \
        -o {FRAMES_DIR}_upscaled \
        --fp32

    # Заменяем оригинальные кадры апскейленными
    !rm -rf {FRAMES_DIR}
    !mv {FRAMES_DIR}_upscaled {FRAMES_DIR}

    # Переименовываем обратно (Real-ESRGAN добавляет суффикс)
    import glob
    upscaled_files = sorted(glob.glob(f"{FRAMES_DIR}/*_out.png"))
    for i, filepath in enumerate(upscaled_files):
        os.rename(filepath, f"{FRAMES_DIR}/{i}.png")

    print("✓ Апскейл завершен!")
else:
    print("⚠️ Upscale модель не найдена, пропускаем апскейл")

# === RIFE ИНТЕРПОЛЯЦИЯ ===
if USE_RIFE:
    print(f"\n🎞️ Интерполяция {NUM_FRAMES} → {NUM_FRAMES * (2**RIFE_EXP)} кадров с RIFE...")
else:
    print(f"\n⚠️ RIFE отключен, создаем видео из {NUM_FRAMES} кадров...")
    !ffmpeg -framerate {FPS} -i {FRAMES_DIR}/%d.png \
        -c:v libx264 -pix_fmt yuv420p -preset fast \
        {WORKSPACE}/GOJO_OUTPUT.mp4 -y -loglevel error

    final_video = f"{WORKSPACE}/GOJO_OUTPUT.mp4"
    if os.path.exists(final_video):
        file_size = os.path.getsize(final_video) / 1024 / 1024
        print(f"✅ ГОТОВО! ({file_size:.1f} MB)")
        display(FileLink(final_video))

if USE_RIFE:
    # Устанавливаем Practical-RIFE
    !rm -rf {WORKSPACE}/RIFE
    !git clone https://github.com/hzwer/Practical-RIFE {WORKSPACE}/RIFE
    %cd {WORKSPACE}/RIFE

    # Устанавливаем зависимости RIFE
    !pip install -q scikit-video

    # Скачиваем модель RIFE v4.26
    print("Скачивание модели RIFE v4.26...")
    !pip install -q gdown
    !mkdir -p train_log
    !gdown 1gViYvvQrtETBgU1w8axZSsr7YUuw31uy -O train_log.zip

    model_downloaded = False

    if os.path.exists("train_log.zip") and os.path.getsize("train_log.zip") > 1000000:
        !unzip -q train_log.zip

        # Ищем модель
        if os.path.exists("train_log/flownet.pkl"):
            model_downloaded = True
            print("✓ Модель RIFE готова!")
        else:
            # Ищем в других местах
            result = !find . -name "flownet.pkl" | head -1
            if result:
                !cp {result[0]} train_log/flownet.pkl
                model_downloaded = True
                print("✓ Модель RIFE готова!")

    if not model_downloaded:
        print("⚠️ Не удалось скачать модель RIFE")
        print("Создаем простое видео из кадров...")
        !ffmpeg -framerate {FPS} -i {FRAMES_DIR}/%d.png \
            -c:v libx264 -pix_fmt yuv420p -preset fast \
            {WORKSPACE}/GOJO_OUTPUT.mp4 -y -loglevel error

        final_video = f"{WORKSPACE}/GOJO_OUTPUT.mp4"
        if os.path.exists(final_video):
            file_size = os.path.getsize(final_video) / 1024 / 1024
            print(f"✅ ГОТОВО! ({file_size:.1f} MB)")
            display(FileLink(final_video))
    else:
        # Запускаем RIFE интерполяцию
        print(f"Запуск интерполяции (exp={RIFE_EXP})...")

        !python inference_video.py \
            --img {FRAMES_DIR} \
            --exp {RIFE_EXP} \
            --output {WORKSPACE}/GOJO_OUTPUT.mp4 \
            --UHD

        # Проверяем результат
        final_video = f"{WORKSPACE}/GOJO_OUTPUT.mp4"
        if os.path.exists(final_video):
            file_size = os.path.getsize(final_video) / 1024 / 1024
            fps = NUM_FRAMES * (2**RIFE_EXP) / 30  # Примерная длина видео
            print(f"\n🎉 ГОТОВО! Видео {NUM_FRAMES * (2**RIFE_EXP)} кадров ({file_size:.1f} MB)")
            display(FileLink(final_video))
        else:
            print("⚠️ RIFE не создал видео, создаем fallback...")
            !ffmpeg -framerate {FPS} -i {FRAMES_DIR}/%d.png \
                -c:v libx264 -pix_fmt yuv420p -preset fast \
                {WORKSPACE}/GOJO_OUTPUT.mp4 -y -loglevel error

            if os.path.exists(final_video):
                file_size = os.path.getsize(final_video) / 1024 / 1024
                print(f"✅ ГОТОВО! ({file_size:.1f} MB)")
                display(FileLink(final_video))

print("\n" + "="*60)
print("📋 ИТОГИ:")
print(f"   Режим: {'AnimateDiff (анимация)' if USE_ANIMATEDIFF else 'Статичные кадры'}")
print(f"   Промпт: {PROMPT[:50]}...")
print(f"   Размер: {WIDTH}x{HEIGHT}")
print(f"   Кадров: {NUM_FRAMES}" + (f" → {NUM_FRAMES * (2**RIFE_EXP)}" if USE_RIFE else ""))
print(f"   Время генерации: {total_gen_time:.1f}s")
if USE_ANIMATEDIFF:
    print("\n💡 Советы для анимации:")
    print("   • Добавьте в промпт: 'turning head', 'blinking', 'hair flowing'")
    print("   • Используйте: 'smooth motion', 'cinematic camera movement'")
    print("   • В negative: 'static, frozen, choppy animation'")
print("="*60)
